<a href="https://colab.research.google.com/github/gsowndarya/ChatGPT-NLP/blob/main/streamlit_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving best_model.h5 to best_model.h5
Saving cleaned_reviews.csv to cleaned_reviews.csv
Saving tokenizer.pkl to tokenizer.pkl


In [2]:
!pip install pyngrok==5.2.1 streamlit


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 kB 14.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 107.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 109.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.1 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19772 sha256=de7593f4022b115ae094112f9bc7d25da2375713bf3538683477d75c24471ca9
  Stored in directory: /root/.cache/pip/wheels/06/70/ba/c8ce0a13cae74dc3a49d7c4155691aa96ce9c2af5c70e809aa
Successfully built pyngrok


In [3]:
!pip install vaderSentiment==3.3.2 contractions==0.1.73

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.3/118.3 kB 5.6 MB/s eta 0:00:00


In [4]:
!pip install nltk

In [5]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [11]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import pickle
import re
import string
import contractions
from collections import Counter
import nltk
from nltk.corpus import stopwords
#from tensorflow.keras.preprocessing.sequence import pad_sequences
#from tensorflow.keras.preprocessing.text import Tokenizer
#from keras.preprocessing.text import Tokenizer
#from keras.models import load_model
from tensorflow.keras.models import load_model
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud

# Download NLTK stopwords if not already
nltk.download('stopwords')

# Set Streamlit page config
st.set_page_config(page_title="ChatGPT Sentiment Dashboard & Analyzer", layout="wide")

# Title
st.title("📊 ChatGPT Review Sentiment Dashboard & Analyzer")

# Load tokenizer and model for sentiment analyzer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

model = load_model('best_model.h5')

stop_words = set(stopwords.words('english'))
max_len = 100

def clean_text(text):
    text = str(text).lower()
    text = contractions.fix(text)
    text = re.sub(' +', ' ', text)
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub(r'\n', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub(r'[^a-z\s]', '', text)
    words = text.split()
    words = [word for word in words if word not in stop_words]
    return ' '.join(words)

def predict_sentiment(text):
    cleaned = clean_text(text)
    seq = tokenizer.texts_to_sequences([cleaned])
    padded = pad_sequences(seq, maxlen=max_len, padding='post')
    pred = model.predict(padded)
    label = np.argmax(pred, axis=1)[0]
    label_map = {0: 'Negative 😞', 1: 'Neutral 😐', 2: 'Positive 😊'}
    return label_map[label]

def show_colored_sentiment(label):
    colors = {
        'Negative 😞': 'red',
        'Neutral 😐': 'orange',
        'Positive 😊': 'green'
    }
    color = colors.get(label, 'black')
    st.markdown(f"<h3 style='color:{color}'>Predicted Sentiment: <b>{label}</b></h3>", unsafe_allow_html=True)

# Sidebar menu
st.sidebar.title("Menu")
section = st.sidebar.radio("Choose Section", ["💬 Sentiment Analyzer", "📈 Data Visualizations"])

if section == "💬 Sentiment Analyzer":
    st.header("🧪 ChatGPT Feedback Analysis")
    user_input = st.text_area("✍️Enter a review about ChatGPT and instantly discover whether it’s Positive, Neutral, or Negative!")

    if st.button("Analyze"):
        if user_input.strip() == "":
            st.warning("Please enter a review.")
        else:
            sentiment = predict_sentiment(user_input)
            show_colored_sentiment(sentiment)

elif section == "📈 Data Visualizations":
    st.header("📈 ChatGPT Review Sentiment Dashboard")

    # Load pre-cleaned CSV file - update the path if needed
    csv_file_path = "cleaned_reviews.csv"
    try:
        df = pd.read_csv(csv_file_path)
        df.rename(columns=lambda x: x.strip().lower().replace(" ", "_"), inplace=True)
        if 'date' in df.columns:
            df['date'] = pd.to_datetime(df['date'], errors='coerce')
    except Exception as e:
        st.error(f"Error loading file: {e}")
        st.stop()

    # Numbered visualization questions dictionary
    question_options = {
        "Q1. Overall Sentiment of User Reviews": "sentiment_distribution",
        "Q2. Sentiment Variation by Rating": "sentiment_by_rating",
        "Q3. Keywords by Sentiment": "sentiment_keywords",
        "Q4. Sentiment Trend Over Time (Monthly)": "sentiment_over_time",
        "Q5. Sentiment by Verified Purchase": "verified_sentiment",
        "Q6. Review Length vs Rating": "review_length_vs_rating",
        "Q7. Which locations show the most Positive or Negative Sentiment?": "location_sentiment",
        "Q8. Sentiment by Platform": "platform_sentiment",
        "Q9. Sentiment by ChatGPT Version": "version_sentiment",
        "Q10. Common Negative Feedback Themes": "negative_feedback_themes"
    }

    selected_question = st.selectbox("Select a question to visualize", list(question_options.keys()))
    selected_viz = question_options[selected_question]

    if selected_viz == "sentiment_distribution":
      st.subheader("Q1. Overall Sentiment of User Reviews")

      sentiment_counts = df['sentiment'].value_counts()

      # Show count table only
      st.table(pd.DataFrame({'Count': sentiment_counts}))

      # Bar chart with Plotly
      fig_bar = px.bar(
          x=sentiment_counts.index,
          y=sentiment_counts.values,
          labels={'x': 'Sentiment', 'y': 'Count'},
          title='Sentiment Counts Bar Chart',
          color=sentiment_counts.index,
          color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'})
      st.plotly_chart(fig_bar)

      # Pie chart with Plotly
      fig_pie = px.pie(
          values=sentiment_counts.values,
          names=sentiment_counts.index,
          title='Sentiment Distribution Pie Chart',
          color=sentiment_counts.index,
          color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'})
      st.plotly_chart(fig_pie)


    elif selected_viz == "sentiment_by_rating":
      st.subheader("Q2. Sentiment Variation by Rating")

      # Sentiment distribution by rating
      sentiment_rating_counts = df.groupby(['rating', 'sentiment']).size().reset_index(name='count')

      fig = px.bar(
          sentiment_rating_counts,
          x='rating',
          y='count',
          color='sentiment',
          title='Sentiment Distribution by Rating',
          labels={'count': 'Number of Reviews', 'rating': 'Rating', 'sentiment': 'Sentiment'},
          color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'},
          barmode='stack')
      st.plotly_chart(fig)

      # Define expected sentiment by rating
      def is_mismatch(row):
        if row['rating'] == 5 and row['sentiment'] != 'Positive':
          return True
        elif row['rating'] == 4 and row['sentiment'] not in ['Positive', 'Neutral']:
          return True
        elif row['rating'] == 3 and row['sentiment'] not in ['Positive', 'Neutral']:
          return True
        elif row['rating'] == 2 and row['sentiment'] not in ['Negative', 'Neutral']:
          return True
        elif row['rating'] == 1 and row['sentiment'] != 'Negative':
          return True
        return False


      # Apply mismatch filter
      mismatch = df[df.apply(is_mismatch, axis=1)]

      st.write(f"Number of mismatched reviews (rating vs sentiment): {len(mismatch)}")

      if not mismatch.empty:
        st.dataframe(mismatch[['rating', 'sentiment', 'review']].head(10))


    elif selected_viz == "sentiment_keywords":
      st.subheader("Q3. Keywords by Sentiment")

      # Word Cloud for each sentiment
      sentiments = ['Positive', 'Neutral', 'Negative']
      for sent in sentiments:
        st.markdown(f"### {sent} Reviews")
        text = " ".join(df[df['sentiment'] == sent]['lemmatized_review'])

        wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(text)

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis("off")
        st.pyplot(fig)

      # Function to get top words
      def get_top_words(sentiment, n=10):
        text = " ".join(df[df['sentiment'] == sentiment]['lemmatized_review'])
        tokens = text.split()
        most_common = Counter(tokens).most_common(n)
        return pd.DataFrame(most_common, columns=['Word', 'Frequency'])

      # Display top 10 words as table
      st.subheader("Top 10 Words by Sentiment")
      for sent in sentiments:
        st.markdown(f"### {sent} Sentiment")
        st.table(get_top_words(sent))


    elif selected_viz == "sentiment_over_time":
        st.subheader("Q4. Sentiment Trend Over Time (Monthly)")

        # Ensure 'date' is datetime and extract 'month'
        df['date'] = pd.to_datetime(df['date'])
        df['month'] = df['date'].dt.to_period('M').astype(str)

        # Group by month and sentiment
        monthly_sentiment = df.groupby(['month', 'sentiment']).size().reset_index(name='count')

        # Plot
        fig = px.line(monthly_sentiment,
                      x='month',
                      y='count',
                      color='sentiment',
                      title='Sentiment Trend Over Time',
                      markers=True,
                      color_discrete_map={
                          'Positive': 'green',
                          'Neutral': 'orange',
                          'Negative': 'red'})

        fig.update_layout(xaxis_title='Month', yaxis_title='Number of Reviews', hovermode='x unified')
        st.plotly_chart(fig)


    elif selected_viz == "verified_sentiment":
        st.subheader("Q5. Sentiment by Verified Purchase")
        # Group and count reviews by verified_purchase and sentiment
        verified_sentiment = df.groupby(['verified_purchase', 'sentiment']).size().reset_index(name='count')

        # Pivot table for display
        pivot_df = verified_sentiment.pivot(index='verified_purchase', columns='sentiment', values='count').fillna(0).astype(int)

        # Calculate difference row (Yes - No)
        diff_row = pivot_df.loc['Yes'] - pivot_df.loc['No']
        diff_row.name = 'Difference (Yes - No)'

        # Append difference row to pivot_df
        pivot_with_diff = pd.concat([pivot_df, diff_row.to_frame().T])

        # Bar chart using Plotly
        fig = px.bar(
            verified_sentiment,
            x='verified_purchase',
            y='count',
            color='sentiment',
            barmode='group',
            title='Sentiment Distribution by Verified Purchase',
            color_discrete_map={
                'Positive': 'green',
                'Neutral': 'orange',
                'Negative': 'red'},
            labels={'verified_purchase': 'Verified Purchase','count': 'Number of Reviews'})

        st.plotly_chart(fig)

        # Optional: Display table
        st.markdown("#### Sentiment Count Table by Verified Purchase")
        st.table(pivot_with_diff)


    elif selected_viz == "review_length_vs_rating":
        st.subheader("Q6. Review Length vs Rating")

        # Group by sentiment and calculate average review length
        avg_length = df.groupby('sentiment')['review_length'].mean().reset_index()

        # Color mapping
        color_map = {'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'}
        colors = [color_map[sent] for sent in avg_length['sentiment']]

        # Plot
        fig = px.bar(avg_length,
                     x='sentiment',
                     y='review_length',
                     color='sentiment',
                     color_discrete_map=color_map,
                     labels={'review_length': 'Average Review Length'},
                     title='Average Review Length by Sentiment')

        st.plotly_chart(fig)


    elif selected_viz == "location_sentiment":
        st.subheader("Q7. Which locations show the most Positive or Negative Sentiment?")
        # Group by location and sentiment to count reviews
        loc_sentiment = df.groupby(['location', 'sentiment']).size().reset_index(name='count')

        # Sort to show top contributing locations (optional)
        top_locations = df['location'].value_counts().head(10).index.tolist()
        loc_sentiment = loc_sentiment[loc_sentiment['location'].isin(top_locations)]

        # Plot
        fig = px.bar(loc_sentiment,
                     x='location',
                     y='count',
                     color='sentiment',
                     color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'},
                     title='Sentiment by Location (Top 10)',
                     labels={'count': 'Review Count', 'location': 'Location'},
                     barmode='group')

        st.plotly_chart(fig)

        sentiment_table = df[df['location'].isin(top_locations)].pivot_table(
            index='location',
            columns='sentiment',
            values='review',
            aggfunc='count',
            fill_value=0).reset_index()

        # Reorder columns for clarity
        sentiment_table = sentiment_table[['location', 'Positive', 'Neutral', 'Negative']]

        # Show the table
        st.dataframe(sentiment_table)


    elif selected_viz == "platform_sentiment":
        st.subheader("Q8. Sentiment by Platform")
        # Count of sentiments per platform
        platform_sentiment = df.groupby(['platform', 'sentiment']).size().reset_index(name='count')
        pivot_platform = platform_sentiment.pivot(index='sentiment', columns='platform', values='count').fillna(0).astype(int)

        # Add a row with difference (Positive - Negative) to understand imbalance
        pivot_platform['Difference (Mobile - Web)'] = pivot_platform.get('Mobile', 0) - pivot_platform.get('Web', 0)

        # Show table
        st.dataframe(pivot_platform.reset_index())

        # Plot bar chart using Plotly
        fig = px.bar(platform_sentiment,
                     x='platform',
                     y='count',
                     color='sentiment',
                     barmode='group',
                     color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'},
                     title="Sentiment Distribution by Platform")

        st.plotly_chart(fig)


    elif selected_viz == "version_sentiment":
        st.subheader("Q9. Sentiment by ChatGPT Version")
        # Group by version and sentiment
        version_sentiment = df.groupby(['version', 'sentiment']).size().reset_index(name='count')

        # Pivot for table view
        pivot_version = version_sentiment.pivot(index='version', columns='sentiment', values='count').fillna(0).astype(int)

        # Add total reviews and most frequent sentiment per version
        pivot_version['Total Reviews'] = pivot_version.sum(axis=1)
        pivot_version['Top Sentiment'] = pivot_version[['Positive', 'Neutral', 'Negative']].idxmax(axis=1)

        st.dataframe(pivot_version.reset_index())

        # Plotly stacked bar chart
        fig = px.bar(version_sentiment,
                     x='version',
                     y='count',
                     color='sentiment',
                     title='Sentiment Distribution by Version',
                     color_discrete_map={'Positive': 'green', 'Neutral': 'orange', 'Negative': 'red'})
        st.plotly_chart(fig)


    elif selected_viz == "negative_feedback_themes":
        st.subheader("Q10. Common Negative Feedback Themes")
        # Filter negative reviews
        negative_reviews = df[df['sentiment'] == 'Negative']['lemmatized_review']

        # Join all reviews and split into tokens
        negative_text = " ".join(negative_reviews).split()

        # Get most common keywords
        top_negative_words = Counter(negative_text).most_common(20)
        top_negative_df = pd.DataFrame(top_negative_words, columns=['Keyword', 'Frequency'])

        st.markdown("### Top 20 Keywords in Negative Reviews")
        st.table(top_negative_df)

        # WordCloud for visualization
        wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='Reds').generate(" ".join(negative_text))

        fig, ax = plt.subplots(figsize=(10, 5))
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis("off")
        st.pyplot(fig)


Overwriting app.py


In [7]:
# Download and install the latest ngrok
!wget -qO ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.zip
!unzip -o ngrok.zip -d /usr/local/bin && chmod +x /usr/local/bin/ngrok

# Add your ngrok authentication token (replace below with your token)
!ngrok config add-authtoken 2xJRPyoirSy9D8YpdP88rm6tWgH_6ocNEUgvkuU6Zyh5xn8NS

Archive:  ngrok.zip
  inflating: /usr/local/bin/ngrok    
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [8]:
import os
import time
import requests
import threading

def run_streamlit():
    os.system("streamlit run app.py --server.headless true")

# Start Streamlit app in a background thread
threading.Thread(target=run_streamlit).start()
time.sleep(5)  # wait for Streamlit to start

# Start ngrok tunnel in background
os.system("ngrok http 8501 &")
time.sleep(5)  # wait for ngrok to start

# Get the public URL
try:
    tunnels = requests.get("http://localhost:4040/api/tunnels").json()
    public_url = tunnels['tunnels'][0]['public_url']
    print("🌐 Your Streamlit app is live here:")
    print(public_url)
except Exception as e:
    print("❌ Error getting ngrok URL:", e)

🌐 Your Streamlit app is live here:
https://218d-35-230-94-24.ngrok-free.app


In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
import pickle

# Example: create a tokenizer and fit on some texts
texts = ['this is a test', 'another test']
tokenizer = Tokenizer(num_words=100)
tokenizer.fit_on_texts(texts)

# Save tokenizer to file (overwrites old tokenizer.pkl)
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

In [10]:
import pickle

with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

# Test tokenizer
print(tokenizer.texts_to_sequences(['this is a test']))

[[2, 3, 4, 1]]
